In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
bact_diag =  pd.read_csv("pn_bact/sofa_diagnoses.csv", index_col=0)
bact_diag["label"] = "no"
virus_diag =  pd.read_csv("pn_virus/sofa_diagnoses.csv", index_col=0)
virus_diag["label"] = "yes"
diag = pd.concat((bact_diag,virus_diag),0)

/tmp/ipykernel_920862/3103570732.py:5: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  diag = pd.concat((bact_diag,virus_diag),0)


In [60]:
cxr_date = pd.read_csv("mimic-iv-cxr-2/mimic-cxr-2.0.0-metadata.csv",)
admission = pd.read_csv("mimic-iv-2/hosp/admissions.csv")

admission = pd.merge(admission, diag.loc[:,["hadm_id","label"]].drop_duplicates())
admission["StartDate"] = [int(i[0:11].replace("-", "")) for i in admission["admittime"]]
admission["EndDate"] = [int(i[0:11].replace("-", "")) for i in admission["dischtime"]]
admission = admission.loc[:,["subject_id", "hadm_id", "StartDate", "EndDate", "label"]]
cxr_date = pd.merge(cxr_date, diag[["subject_id"]].drop_duplicates())
cxr_date = cxr_date.loc[:,["subject_id","study_id","StudyDate"]].drop_duplicates()
cxr_hadm = pd.merge(admission, cxr_date)
cxr_hadm = cxr_hadm.loc[(cxr_hadm["StudyDate"]-cxr_hadm["StartDate"]>-3) & (cxr_hadm["StudyDate"]-cxr_hadm["EndDate"]<3)]

In [55]:
cxrlist = pd.read_csv("mimic-iv-cxr-2/cxr-record-list.csv")
report_train = pd.read_csv("mimic-iv-cxr-2/mimic_train_impressions.csv", index_col=0)
report_test = pd.read_csv("mimic-iv-cxr-2/mimic_test_impressions.csv",)
cxrreport = pd.concat((report_train,report_test),0)
cxrreports = pd.merge(cxrlist,cxrreport).drop_duplicates()
diag_cxr = pd.merge(cxr_hadm,cxrreport).drop_duplicates()

In [68]:
df = diag_cxr
train_set = {
    "data": [{
        'paragraphs': [],
        'title': "LungB"
    }],
    "version": "LungB"
}
train_qa = []
im = {"yes": False, "no": True}

df = df.loc[df["report"].isnull()==False, :]
for idx in df.index:
    
    qa = {
        'qas': [{
            'id': str(df.loc[idx, "study_id"]),
            'question': "Is patient's cxr report consistent with viral infection?",
            'is_impossible': im[df.loc[idx, "label"]],
            'answers': df.loc[idx, "label"]
        }],
        'context': df.loc[idx, "report"]
    }
    train_qa.append(qa)

train_set['data'][0]["paragraphs"] = train_qa
json.dump(train_set, open("mimic-iv-cxr-2/train_set.json", "w"))